In [1]:
import numpy as np

import numba

import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

from auxiliary import *

### Geometric properties

In [ ]:
# length of the rod
length = 0.1

# diameter
radius = 0.0005

# cross-section area
A = np.pi * radius**2

# second moment of area
I = np.pi * radius**4 / 4

### Material properties

In [ ]:
# E-modulus
E = 200e9

# shear modulus
G =  85e9

#?? names
GA = G * A
EA = E * A
EI = E * I
GJ = 2 * G * I

# elasticity tensor
C = np.diag([GA, GA, EA, EI, EI, GJ])

### Dicretization properties

In [ ]:
# number of elements
n_el = 100

# number of nodes
n_n = n_el + 1

# length of each element
h = np.ones((n_el)) * (length / n_el)

### Solution variables

Declaration in global namespace and initialization

In [ ]:
# centerline displacement at each node
centerline = np.zeros((3, n_n))
for i in range(n_el):
    centerline[2, i+1] = centerline[2, i] + h[i]

# rotation vector at each node
rotation = np.zeros((3, n_n))

In [ ]:
plot_centerline(centerline)

### Boundary conditions??

In [ ]:
# force boundary condition at s = 0
#?? Dirichlet boundary
b0 = np.array([0, 0, 0, 0, 0, 0])

# force boundary condition at s = 1
# ??unknown transverse load
b1 = np.array([0, 0, 0, 0, 0, 0])

### ???

In [ ]:
# displacement increment vector
increments = np.zeros((6*(n_n), 1))

In [ ]:
# curvature vector in each element
kappa = np.zeros((3, n_el))

### 

In [ ]:
def jacobmat():

    # global Jacobian matrix
    jac = np.zeros((6*n_n, 6*n_n))
    
    # global residual vector
    residual = np.zeros((6*n_n))
    
    # elasticity tensor
    C = np.zeros((6,6))
    
    # rotation vector
    theta = np.zeros((3))
    
    # incremental rotation tangent vector
    delthp = np.zeros((3))
    
    # internal force  vector in inertial frame
    N = np.zeros((3))
    
    # internal moment vector
    M = np.zeros((3))
    
    # translational displacement tangent vector
    phip = np.zeros((3))
    
    # strain vector
    gama = np.zeros((3))
    
    # skew symmetric matrix of N
    Nmat = np.zeros((3,3))
    
    # skew symmetric matrix of M
    Mmat = np.zeros((3,3))
    
    # skew symmetric matrix of phip
    phimat = np.zeros((3,3))
    
    # rotation matrix
    rotmat = np.zeros((3,3))
    
    # increment of rotation matrix
    delrotmat = np.zeros((3,3))



    # calculation of Jacobian and residual vector

    for i in range(n_el):
        
        # update of configuration variables
        centerline[:, i] = centerline[:, i] + increments[6*i:6*i+2]
        
        # finding rotmat corresponding to infinitesimal rotation
        # delth = incremental rotation vector
        delth = increments[6*i+3:6*i+6]
        
        delrotmat = rotation_matrix_from_axis_anlge(delth)

        # finding rotmat corresponding to previous axial vector
        theta = rotation[:, i]
        
        rotmat = rotation_matrix_from_axis_anlge(theta)
    
        # total rotation
        rotmat = np.dot(delrotmat, rotmat)

        # finding updated axial vector corresponding to total rotation
        cand = np.array([np.trace(rotmat), rotmat[0,0], rotmat[1,1], rotmat[2,3]])
        ind = np.argmax(cand)
        maxval = cand[ind]
        if ind ==0:
            qq[0] = np.sqrt(1 + maxval) * 0.5
            qq[1] = (rotmat[2, 1] - rotmat[1, 2]) / (4*qq[0])
            qq[2] = (rotmat[0, 2] - rotmat[2, 0]) / (4*qq[0])
            qq[3] = (rotmat[1, 0] - rotmat[0, 1]) / (4*qq[0])
        elif ind==1:
            qq[1] = np.sqrt(0.5*maxval + 0.25*(1 - np.trace(rotmat)))
            qq[0] = (rotmat[2, 1] - rotmat[1, 2]) / (4*qq[1])
            qq[2] = (rotmat[1, 0] + rotmat[0, 1]) / (4*qq[1])
            qq[3] = (rotmat[2, 0] + rotmat[0, 2]) / (4*qq[1])
        elif ind==2:
            qq[2] = np.sqrt(0.5*maxval + 0.25*(1 - np.trace(rotmat)))
            qq[0] = (rotmat[0, 2] - rotmat[2, 0]) / (4*qq[2])
            qq[1] = (rotmat[0, 1] + rotmat[1, 0]) / (4*qq[2])
            qq[3] = (rotmat[2, 1] + rotmat[1, 2]) / (4*qq[2])
        elif ind==3:
            qq[3] = np.sqrt(0.5*maxval + 0.25*(1 - np.trace(rotmat)))
            qq[0] = (rotmat[1,0] - rotmat[0,1]) / (4*qq[3])
            qq[1] = (rotmat[0,2] + rotmat[2,0]) / (4*qq[3])
            qq[2] = (rotmat[1,2] + rotmat[2,1]) / (4*qq[3])
        else:
            raise Exception
        
        if qq[0] >= 0:
            normth = 2*np.real(np.arcsin(np.linalg.norm(qq[1:])))
        else:
            normth = 2*(np.pi - np.real(np.arcsin(np.linalg.norm(qq[1:]))))
        
        if normth < 1e-6:
            rotation[:, i] = np.zeros((3))
        else:
            rotation[:, i] = (normth/np.linalg.norm(qq[1:]))*qq[1:]

        

        # calculation of rotation matrix at mid point of element

        theta = (rotation[:, i-1] + rotation(:, i)) / 2
        
        # required because extracted axial vector may differ by 2*pi
        if(np.linalg.norm(rotation[:, i] - rotation[:, i-1]) > np.pi)
            theta = rotation[:, i]
        
        rotmat = rotation_matrix_from_axis_anlge(theta)
        
        

        # update of strain variables at mid point of element

        phip     = (centerline[:, i+1] - centerline[:, i]) / h[i]
        gama     = np.dot(rotmat.T, phip)
        gama[2]  = gama[2] - 1
        
        # update of global kappa
        delthp  = (increments[6*i+3:6*i+6] - increments[6*i-3:6*i]) / h[i]
        delth   = increments[6*i-3:6*i] + (increments[6*i+3:6*i+6] - increments[6*i-3:6*i])/2
        nrm     = np.linalg.norm(delth)
        if nrm < 1e-6:
            delkapa = delthp + 0.5*np.cross(delth, delthp)
        else:
            x = sin(nrm)/nrm
            delu = delth/nrm
            delkapa = x*delthp + ((1-x)*(np.dot(delu.T, delthp)))*delu + (2*(np.sin(0.5*nrm)/nrm)**2)*np.cross(delth, delthp)
        
        kappa[:, i]   = kappa[:, i] + np.dot(rotmat.T, delkapa)

        
        
        
        

        # !!internal reactions in inertial frame
        N  = np.dot(rotmat, [C(1,1)*gama(1);C(2,2)*gama(2);C(3,3)*gama(3)])
        M  = rotmat*[C(4,4)*kapa(1,i);C(5,5)*kapa(2,i);C(6,6)*kapa(3,i)]

        


        # residual and jacobian calculation

        crossphin = np.cross(phip, N)
        
        residue[6*i-6:6*i] = residue[6*i-6:6*i] + np.hstack(-N, -(0.5*h[i])*crossphin - M)
        residue[6*i:6*i+6] = residue[6*i:6*i+6] + np.hstack(+N, -(0.5*h[i])*crossphin + M)

        C11 = np.dot(np.dot(rotmat, C[0:3, 0:3]), rotmat.T)
        C11 = (C11 + C11.T) / 2
        C12 = np.dot(np.dot(rotmat, C[0:3, 3:6]), rotmat.T)
        C21 = C12.T
        C22 = np.dot(np.dot(rotmat, C[3:6, 3:6]), rotmat.T)
        C22 = (C22 + C22.T) / 2

        phimat = skew_matrix_from_vector(phip)
        Nmat = skew_matrix_from_vector(N)
        Mmat = skew_matrix_from_vector(M)


        # material tangent stiffness (symmetric part)

        jac(6*i-5:6*i,6*i-5:6*i)        = jac(6*i-5:6*i,6*i-5:6*i)      + [ C11/h(i)     -.5*(C11*phimat)+C12/h(i); -.5*(phimat'*C11)+C21/h(i)    (phimat'*C11*phimat)*(h(i)/3)-.5*(phimat'*C12+C21*phimat)+C22/h(i)];
        jac(6*i+1:6*i+6,6*i+1:6*i+6)    = jac(6*i+1:6*i+6,6*i+1:6*i+6)  + [ C11/h(i)      .5*(C11*phimat)+C12/h(i);  .5*(phimat'*C11)+C21/h(i)    (phimat'*C11*phimat)*(h(i)/3)+.5*(phimat'*C12+C21*phimat)+C22/h(i)];
        jac(6*i+1:6*i+6,6*i-5:6*i)      = jac(6*i+1:6*i+6,6*i-5:6*i)    + [-C11/h(i)      .5*(C11*phimat)-C12/h(i); -.5*(phimat'*C11)-C21/h(i)    (phimat'*C11*phimat)*(h(i)/6)-.5*(phimat'*C12-C21*phimat)-C22/h(i)];
        jac(6*i-5:6*i,6*i+1:6*i+6)      = jac(6*i-5:6*i,6*i+1:6*i+6)    + [-C11/h(i)     -.5*(C11*phimat)-C12/h(i);  .5*(phimat'*C11)-C21/h(i)    (phimat'*C11*phimat)*(h(i)/6)+.5*(phimat'*C12-C21*phimat)-C22/h(i)];

        # geometric tangent stiffness (non-symmetric)

        jac[6*i-4:6*i, 6*i-4:6*i]       = jac(6*i-5:6*i,6*i-5:6*i)      + [zeros(3,3)    .5*Nmat;    -.5*Nmat    .5*Mmat-(phimat'*Nmat)*(h(i)/3)];
        jac[6*i+0:6*i+6,6*i:6*i+6]    = jac(6*i+1:6*i+6,6*i+1:6*i+6)  + [zeros(3,3)   -.5*Nmat;     .5*Nmat   -.5*Mmat-(phimat'*Nmat)*(h(i)/3)];
        jac[6*i+0:6*i+6,6*i-4:6*i]      = jac(6*i+1:6*i+6,6*i-5:6*i)    + [zeros(3,3)   -.5*Nmat;    -.5*Nmat   -.5*Mmat-(phimat'*Nmat)*(h(i)/6)];
        jac[6*i-4:6*i, 6*i:6*i+6]      = jac(6*i-5:6*i,6*i+1:6*i+6)    + [zeros(3,3)    .5*Nmat;     .5*Nmat    .5*Mmat-(phimat'*Nmat)*(h(i)/6)];

        # tangent due to distributive load

        # tangent due to boundary loads
    

    residue[0:6] = residue[0:6] + b0
    residue[-5:] = residue[-5:] - b1

    # calculation of displacement increment vector

    increments[6:] = np.linalg.solve(-jac[6:, 6:], residue[6:])
    
    increments_norm = norm(increments[6:])
    if increments_norm > 1:
        increments[6:] = increments[6:] / increments_norm
    
    residual_norm = np.linalg.norm(residue[6:])

    return increments_norm, residual_norm

### Main program

In [ ]:
def staticsolver():
    
    # iteration number
    iteration = 0 
    
    # initial perturbation in transverse ??displacement
    #b1[0] = 0
    #desired_load = b1[1]
    #b1[1] = 0
    
    # norm of global incremental displacement vector
    increments_norm = 0
    
    # norm of residual vector
    residual_norm = 0
    
    # empty array for logging transverse load (at s=1, in direction 2)
    load_data = np.array([])
    
    # empty array for logging transverse displacement (at last node, in direction 2)
    displacement_data = np.array([])
    
    
    # Newton-Rhapson iterations
    
    while(b1[1] < 1):
        iteration   = iteration + 1
    
        # display output
        print('increments_norm', increments_norm)
        print('residual_norm', residual_norm)
        print('transverse load', b1[1])
    
        # when converged, change of boundary condition (load control)
        if(increments_norm < 1e-4 and residual_norm < 1e-4):
            
            # logging
            load_data = np.append(load_data, b1[1])
            displacement_data = np.append(displacement_data, z0[1, -1])
            #??col   = [b1[1], z0[1, -1], b1[1]*L**3/(3*EI)]
            
            # increasing transverse load
            b1[1] = b1[1] + 0.1
    
    
        #plot3(z0(3,:),z0(1,:),z0(2,:));
        #pause(.001);
    
        increments_norm, residual_norm = jacobmat()

    #plot(data1(:,1),data1(:,2),'r');
    #plot(data1(:,1),data1(:,3),'b');
    plt.plot(np.arange(len(load_data)), load_data)

In [ ]:
staticsolver()

In [447]:
def test_quaternion_roundtrip():
    """
    Tests the functions `quaternion_from_rotation_matrix` and
    `rotation_matrix_from_quaternion` by creating a random
    unit quaternion and checking if the composition of the
    two functions renders the identity.
    """
    # creating a random unit quaternion
    unit_quaternion = np.random.randn(4)
    unit_quaternion /= np.linalg.norm(unit_quaternion)
    
    # apply the composition of the two functions
    rotation_matrix = rotation_matrix_from_quaternion(unit_quaternion)
    unit_quaternion_test = quaternion_from_rotation_matrix(rotation_matrix)
    
    print(np.linalg.norm(unit_quaternion_test - unit_quaternion))
    
    print(unit_quaternion)
    
    check = np.allclose(unit_quaternion, unit_quaternion_test)
    
    if not check:
        print(unit_quaternion_test)
        
        # assert that the rotation matrix is unitary
        assert np.allclose(np.eye(3), np.dot(rotation_matrix.T, rotation_matrix))
    
    return check

In [536]:
test_quaternion_roundtrip()

2.77555756156e-17
[ 0.1506223   0.83416791  0.46442998 -0.25647927]


True

In [509]:
def test_quaternion_roundtrip2():
    """
    Tests the functions `quaternion_from_rotation_matrix` and
    `rotation_matrix_from_quaternion` by creating a random
    rotation matrix (via the function `rotation_matrix_from_axis_anlge`)
    and checking if the composition of the two functions
    renders the identity.
    """

    # create a random rotation in axis-angle representation
    axis_angle = np.random.randn(3)
    
    # convert to a rotation matrix
    rotation_matrix = rotation_matrix_from_axis_anlge(axis_angle)
    
    # assert that the rotation matrix is unitary
    assert np.allclose(np.eye(3), np.dot(rotation_matrix.T, rotation_matrix))

    # apply the composition of the two functions 
    unit_quaternion = quaternion_from_rotation_matrix(rotation_matrix)
    rotation_matrix_test = rotation_matrix_from_quaternion(unit_quaternion)
    
    #print(np.linalg.norm(rotation_matrix_test - rotation_matrix))

    assert np.allclose(rotation_matrix , rotation_matrix_test)

In [516]:
test_quaternion_roundtrip2()